In [1]:
# standard import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import pyreadr

# sklearn
from sklearn.linear_model import (
    LinearRegression,
    RidgeCV,
    LassoCV,
    ElasticNetCV,
    # HuberRegressor,
    # QuantileRegressor,
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
)
from sklearn.utils import resample
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.datasets import make_regression
from sklearn import datasets

# miscilaneous models
from xgboost import XGBRegressor

# from quantile_forest import RandomForestQuantileRegressor
# from imodels import get_clean_dataset
from joblib import Parallel, delayed

# from exp_utils import *

# from methods import *
import time
from scipy.stats import multivariate_normal
import pickle
import blosc
import os

import itertools
from copy import deepcopy
import warnings

In [2]:
# methods
from pcs_UQ import PCS_UQ
from gamma_algo import *

In [3]:
models = {
            "linear": {"Ridge": RidgeCV(), 
                       "Lasso": LassoCV(max_iter=5000, random_state=777), 
                       "ElasticNet": ElasticNetCV(max_iter=5000, random_state=777)},
            "bagging": {"ExtraTrees": ExtraTreesRegressor(min_samples_leaf = 5, max_features = 0.33, n_estimators = 100, random_state=777), 
                        "RandomForest": RandomForestRegressor(min_samples_leaf = 5, max_features = 0.33, n_estimators = 100, random_state=777)},
            "boosting": {"XGBoost": XGBRegressor(random_state=777), 
                         "AdaBoost": AdaBoostRegressor(random_state=777)}
        }

In [4]:
data = pd.read_csv('Jan_0529_raw.csv')
data.drop(columns=data.columns[:2], axis=1, inplace=True)
data.rename(columns = {'Unnamed: 0' : 'id'}, inplace=True)

In [5]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

In [6]:
data['start'] = pd.to_datetime(data['start'])
data['end'] = pd.to_datetime(data['end'])

# Function to filter data within a given range
def select_data_within_range(df, start_column, start_date, end_date):
    if not pd.api.types.is_datetime64_any_dtype(df[start_column]):
        df[start_column] = pd.to_datetime(df[start_column])

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    return df[(df[start_column] >= start_date) & (df[start_column] <= end_date)]

# Define training and evaluation time periods
train_start, train_end = "2022-01-01 00:00:00", "2022-01-12 22:30:00"
eval_start, eval_end = "2022-01-12 22:30:00", "2022-01-31 23:59:00"

# Split dataset into training and evaluation sets
training_data = select_data_within_range(data, 'start', train_start, train_end)
evaluation_data = select_data_within_range(data, 'start', eval_start, eval_end)

# Select features and target variable
features = training_data[['prev_tput', 'prev_rtt', 'prev_retx_rate', 'interval', 'prev_size', 'size_ratio', 'size_bin', 'ip_subnet', 'overlap']]
encoded_features = pd.get_dummies(features, columns=['size_bin', 'ip_subnet'], drop_first=True)
target = np.log(training_data['tput']).values.ravel()



In [7]:
# Split the dataset into training, validation, and test sets
subgroup_df = features[['ip_subnet', 'size_bin', 'overlap']].copy()

X_train_val, X_test, y_train_val, y_test, subgroup_train_val, subgroup_test = train_test_split(
    encoded_features, target, subgroup_df, test_size=0.2, random_state=77)

X_train, X_val, y_train, y_val, subgroup_train, subgroup_val = train_test_split(
    X_train_val, y_train_val, subgroup_train_val, test_size=0.4, random_state=77)

In [8]:
# Define gamma parameters
gamma_params = {
    "selection_mode": "multiplicative",
    "fit_mode": "vanilla",
    "threshold": None,
    "clip_mode": "no_scale"
    }

pcs_uq_orig = PCS_UQ(models)

# Train PCS_UQ model
results = pcs_uq_orig.train(
    x_train=X_train, 
    x_val=X_val, 
    y_train=y_train, 
    y_val=y_val, 
    n_boot = 15,
    file_name='Jan_0529'
)

# Perform calibration
val_results_df = pcs_uq_orig.calibrate(
    x_val=X_val,
    y_val=y_val,
    gamma_params=gamma_params,
    best="all"
)

# Make predictions on the test set
test_results_df = pcs_uq_orig.predict(x_test=X_test)

# Evaluate the model
pcs_evaluation_results = pcs_uq_orig.evaluate(y_test=y_test)

Evaluation complete, here's the result
  model_group         model           rmse           mae            r2
0      linear         Ridge  788238.964849  49397.453143 -1.078050e+12
1      linear         Lasso       0.488904      0.307635  5.852658e-01
2      linear    ElasticNet       0.488904      0.307635  5.852658e-01
3     bagging    ExtraTrees       0.384512      0.275103  7.434668e-01
4     bagging  RandomForest       0.209085      0.099367  9.241478e-01
5    boosting       XGBoost       0.199353      0.084675  9.310443e-01
6    boosting      AdaBoost       0.745479      0.695041  3.573954e-02
Bootstrapping
processing data


In [10]:
ip_subnet_results = pcs_uq_orig.evaluate_subgroups(y_test, subgroup_test['ip_subnet'])
display(ip_subnet_results)

# Evaluate on size_bin
size_bin_results = pcs_uq_orig.evaluate_subgroups(y_test, subgroup_test['size_bin'])
size_bin_results

,subgroup,coverage,avg_length,median_length,range_y_test,alpha,scaled_avg_length,scaled_median_length
0,131,0.895858,0.714658,0.520271,10.898026,0.1,0.065577,0.047740
1,140,0.750000,6.937067,6.746297,0.135911,0.1,0.636543,0.619038
2,142,1.000000,4.927797,4.059819,0.632313,0.1,0.452173,0.372528
3,205,1.000000,3.366074,3.366074,0.439063,0.1,0.308870,0.308870


,subgroup,coverage,avg_length,median_length,range_y_test,alpha,scaled_avg_length,scaled_median_length
0,large,0.964743,1.085296,1.161868,4.682613,0.1,0.099586,0.106613
1,medium,0.859331,0.552199,0.389321,8.837906,0.1,0.050670,0.035724
2,small,0.866421,0.543439,0.355213,9.084937,0.1,0.049866,0.032594


In [11]:
# Subgroup Calibration
gamma_params = {
    "selection_mode": "multiplicative",
    "fit_mode": "vanilla",
    "threshold": None,
    "clip_mode": "no_scale"
    }

pcs_uq_sg = PCS_UQ(models)

# Train PCS_UQ model
results = pcs_uq_sg.train(
    x_train=X_train, 
    x_val=X_val, 
    y_train=y_train, 
    y_val=y_val, 
    n_boot = 15,
    file_name='Jan_0529'
)

pcs_uq_sg.calibrate_by_subgroup(X_val, y_val, subgroup_val['size_bin'], gamma_params)

pcs_uq_sg.predict_by_subgroup(X_test, subgroup_test['size_bin'])

pcs_uq_sg.evaluate(y_test)

Evaluation complete, here's the result
  model_group         model           rmse           mae            r2
0      linear         Ridge  788238.964849  49397.453143 -1.078050e+12
1      linear         Lasso       0.488904      0.307635  5.852658e-01
2      linear    ElasticNet       0.488904      0.307635  5.852658e-01
3     bagging    ExtraTrees       0.384512      0.275103  7.434668e-01
4     bagging  RandomForest       0.209085      0.099367  9.241478e-01
5    boosting       XGBoost       0.199353      0.084675  9.310443e-01
6    boosting      AdaBoost       0.745479      0.695041  3.573954e-02
Bootstrapping
Processing data...


,coverage,avg_length,median_length,range_y_test,alpha,scaled_avg_length,scaled_median_length
0,0.895323,0.651362,0.509444,10.898026,0.1,0.059769,0.046746
